In [15]:
import gfc
from gfc import *
import numpy as np
import scipy as sp
import copy as cp
from extreme_deconvolution import extreme_deconvolution as xd
import matplotlib.pyplot as plt
from matplotlib import rc
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import Angle
from pygaia.astrometry.vectorastrometry import phaseSpaceToAstrometry, sphericalToCartesian, normalTriad
from pygaia.astrometry.coordinates import CoordinateTransformation, Transformations
from pygaia.astrometry import constants

galtoicrs = CoordinateTransformation(Transformations.GAL2ICRS)
icrstogal = CoordinateTransformation(Transformations.ICRS2GAL)

rc('font',**{'family':'serif', 'serif':['Times']})
rc('text', usetex=True)

root_folder = "/disks/strw9/vanweenen/mrp1/results/simulation"

# Simulate a collection of stars distributed at constant space density around the sun, 
# with distances between 1 and 500 pc. Use Galactic coordinates. Each star is assigned
# a 3D space velocity drawn from a superposition of several 3D normal distributions in 
# velocity space.
rmin = 1. #pc
rmax = 500. #pc
N = 1000 #nr of stars

r = np.random.uniform(rmin**3, rmax**3, N)**(1./3.) #pc
theta = np.arccos(np.random.uniform(-1, 1, N))
phi = np.random.uniform(0, 2*np.pi, N)

x_gal,y_gal,z_gal = sphericalToCartesian(r, phi, theta)

# Define the space velocities (u,v,w), again in galactic coordinates
K = 2 #nr of components
meanstep = 20. #km/s
sigma = 3.
p = 0. #relation between amplitudes of components
q = 0. #relation between sigmas of components

wmin = 0.5 ; wmax=5 ; wstep=.25
Kmin=1 ; Kmax=10 ; Kstep=1

def simulate_amps(k):
    a = np.empty([k])
    rest = 1
    for i in range(k-1):
        a[i] = (1./k)/(i+2)**p
        rest -= a[i]
    a[k-1] = rest
    return a

def simulate_means(k):
    m = np.empty([k, 3])
    for i in range(k):
        m[i,:] = [meanstep*i, meanstep*i, meanstep*i]
    return m

def simulate_covs(k):
    c = np.zeros((k,3,3))
    for i in range(k):
        c[i,0,0] = (sigma*(i**q))**2
        c[i,1,1] = (sigma*(i**q))**2
        c[i,2,2] = (sigma*(i**q))**2
    return c

initamps = []
initmeans = []
initcovs = []

for k in range(Kmin, Kmax+1):
    initamps.append(simulate_amps(k))
    initmeans.append(simulate_means(k))
    initcovs.append(simulate_covs(k))

component = np.random.choice(K, N, p=initamps[K-1]) #choose component
uvw_gal = np.empty([N,3])
for i in range(N):
    uvw_gal[i] = np.random.multivariate_normal(initmeans[K-1][component[i],:], initcovs[K-1][component[i],:,:])

#uvwcov_gal = np.zeros_like(uvw_gal) #use this when not using astrometry coordinate transformation

In [16]:
# Transform the simulated positions and velocities into astrometric observables and radial velocity in ICRS
print "Transformation to astrometric observables.."
x_icrs, y_icrs, z_icrs = galtoicrs.transformCartesianCoordinates(x_gal, y_gal, z_gal)
u_icrs, v_icrs, w_icrs = galtoicrs.transformCartesianCoordinates(uvw_gal[:,0], uvw_gal[:,1], uvw_gal[:,2])
alpha, delta, parallax, mura, mudec, vrad = phaseSpaceToAstrometry(x_icrs, y_icrs, z_icrs, u_icrs, v_icrs, w_icrs) #rad, rad, mas, mas/yr, mas/yr, km/s
alpha = Angle(alpha, u.radian) ; delta = Angle(delta, u.radian)
astr_true = np.column_stack((alpha.mas, delta.mas, parallax, mura, mudec, vrad))

print "Simulate measured values.."
#Simulate measured values using measurement errors
measurement_error = np.array([0.,0.,0.3,1.,1.,0])
measurement_covar = np.diag(measurement_error**2) #measurement errors
astr_measured = np.empty([N,6])
for i in range(N):
    astr_measured[i] = np.random.multivariate_normal(astr_true[i,:], measurement_covar, 1)
errors_measured = np.tile(measurement_error, (N,1))
corr_measured = np.tile(np.tile([0.],15), (N,1))

arr_astr = np.column_stack((astr_measured, errors_measured, corr_measured))
labels = ('ra', 'dec', 'parallax', 'pmra', 'pmdec', 'vrad', 'ra_error', 'dec_error', 'parallax_error', 'pmra_error', 'pmdec_error', 'vrad_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'ra_vrad_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'dec_vrad_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'parallax_vrad_corr', 'pmra_pmdec_corr', 'pmra_vrad_corr', 'pmdec_vrad_corr')
t_astr=Table(arr_astr, names=labels)

print "Projection.."

# Calculate the projection matrix analogous to equation (1) in Bovy et al 2009 
# (https://ui.adsabs.harvard.edu/#abs/2009ApJ...700.1794B/abstract). Note the different ordering of the
# velocity components.
for i in ["ra", "dec"]:
    gfc.add_rad(t_astr,i)
matrix.transformation(t_astr)#, vrad_col = 'vrad')
warr = gfc.XD_arr(t_astr, "w")
wcovar = gfc.XD_arr(t_astr, "S") ; wcovar[:,0,0] = 1e15
proj = gfc.XD_arr(t_astr, "R")
uvwarr = gfc.XD_arr(t_astr, "UVW")

Transformation to astrometric observables..
Simulate measured values..
Projection..
Transforming to radians for..  ra
Transforming to radians for..  dec


ValueError: Illegal type <type 'NoneType'> for table item access

In [ ]:
# Perform XD
print "Perform XD.."
# The input to XD are the values of v_alpha*, v_delta, vrad. The other input required is the projection matrix.
# The values of v_alpha*, v_delta, vrad are obtained from (alpha, delta, parallax, mura,  mudec, vrad).
dim = 3.
wrange = np.arange(wmin, wmax + wstep, wstep)**2.
Krange = range(Kmin, Kmax + Kstep, Kstep)

logL = np.tile(np.nan, (len(Krange), len(wrange)))
AIC = np.tile(np.nan, (len(Krange), len(wrange)))
MDL = np.tile(np.nan, (len(Krange), len(wrange)))

max_logL = -1e15
min_AIC = 1e15
min_MDL = 1e15

for k in Krange:
    print k
    for w in xrange(len(wrange)):
        amps_xd, means_xd, covs_xd, logL[k-1,w] = XD(warr, wcovar, initamps[k-1], initmeans[k-1], initcovs[k-1], projection=proj, w=wrange[w]) #extreme deconvolution
        logL[k-1,w] *= N
        AIC[k-1,w] = -2./N*(N-1.-dim-Kmax/2.)*logL[k-1,w]+3*k*dim
        MDL[k-1,w] = -logL[k-1,w] + k*dim*np.log(N)/2.
        if logL[k-1,w] > max_logL:
            max_logL = logL[k-1,w]
            bestK_logL = k ; bestw_logL = wrange[w]
            amps_logL = amps_xd ; means_logL = means_xd ; covs_logL = covs_xd
        if AIC[k-1,w] < min_AIC:
            min_AIC = AIC[k-1,w]
            bestK_AIC = k ; bestw_AIC = wrange[w]
            amps_AIC = amps_xd ; means_AIC = means_xd ; covs_AIC = covs_xd
        if MDL[k-1,w] < min_MDL:
            min_MDL = MDL[k-1,w]
            bestK_MDL = k ; bestw_MDL = wrange[w]
            amps_MDL = amps_xd ; means_MDL = means_xd ; covs_MDL = covs_xd

print "Best values.."
print "logL: ", "best K = {0} ; best w = {1}".format(bestK_logL, bestw_logL)
print amps_logL, means_logL, covs_logL
print "AIC: ", "best K = {0} ; best w = {1}".format(bestK_AIC, bestw_AIC)
print amps_AIC, means_AIC, covs_AIC
print "MDL: ", "best K = {0} ; best w = {1}".format(bestK_MDL, bestw_MDL)
print amps_MDL, means_MDL, covs_MDL
print "Done"

In [ ]:
# Plot output data

def hist_data(i, x, amps, means, covs, c, l):
    norm = np.zeros(len(x))
    for j in range(len(amps)):
        norm += amps[j]*sp.stats.norm.pdf(x, loc=means[j,i], scale=np.sqrt(covs[j,i,i]))
    ax[i].plot(x, norm, label=l, color=c, lw=1)

names = ['U', 'V', 'W']
unit = ' (km/s)'
fig, ax = plt.subplots(1,3,sharey=True,figsize=(10,3))
for i in range(0,3):
    x = np.linspace(np.amin(uvw_gal[:,i]), np.amax(uvw_gal[:,i]), N)
    hist_data(i, x, initamps[K-1], initmeans[K-1], initcovs[K-1], "blue", "input")
    hist_data(i, x, amps_logL, means_logL, covs_logL, "red", "XD")
    hist_data(i, x, amps_AIC, means_AIC, covs_AIC, "purple", "AIC")
    hist_data(i, x, amps_MDL, means_MDL, covs_MDL, "green", "MDL")
    ax[i].hist(uvw_gal[:,i], bins=50, normed=True, facecolor='blue', histtype='stepfilled', alpha=0.4)
    ax[i].set_xlabel(names[i] + unit)
    ax[i].legend(loc="upper right", prop={'size': 6})
plt.suptitle('Histogram of velocity in Carthesian coordinates with K = {0}, N = {1}'.format(K,N))
print "Saving figure.."
plt.savefig(root_folder + '/hist_velocity_K{0}_N{1}'.format(K,N) + '.png')
plt.show()
print "Done"

In [ ]:
# Plot log likelihood, AIC and MDL for different K and w
print "Starting"

title = ("Log. likelihood","AIC" ,"MDL")
test = (logL, AIC, MDL)
bestK = (bestK_logL, bestK_AIC, bestK_MDL)
bestw = (bestw_logL, bestw_AIC, bestw_MDL)
c = ("black", "white", "white")

fig, ax = plt.subplots(1,3,figsize=(10,3))
for i in range(3):
    ax[i].set_xlabel("$K$") ; ax[i].set_ylabel("$\sqrt{w}$ (km s$^{-1}$)")
    ax[i].set_xlim(Kmin-.5, Kmax+.5) ; ax[i].set_ylim(wmin-.125, wmax+.125)
    ax[i].set_title(title[i] + " for K={0}, N={1}".format(K,N))
    ax[i].scatter(bestK[i], np.sqrt(bestw[i]), color=c[i], marker="o", s=150)
    im = ax[i].imshow(test[i].T, cmap=plt.cm.viridis, extent=(Kmin-.5, Kmax+.5, wmin-.125, wmax+.125), aspect='auto', interpolation='none', origin="lower")
    cbar = fig.colorbar(im, ax=ax[i], orientation="vertical")

print "Saving figure.."
plt.savefig(root_folder + '/logL-Kw_K{0}_N{1}'.format(K,N) + '.png')
plt.show()
print "Done"